# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [110]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint

# Import API key
from api_keys import g_key

#configure map
gmaps.configure(api_key = g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [111]:
#load csv and DF
vacationDF = pd.read_csv("weatherdata.csv")
vacationDF.drop(vacationDF.columns[vacationDF.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
vacationDF.head()

City  Latitude  Longitude  Temperature  Cloudiness  Wind Speed  \
0   Arraial do Cabo    -22.97     -42.03        77.00          90        3.36   
1           Rikitea    -23.12    -134.97        75.11          54       11.03   
2        Bredasdorp    -34.53      20.04        50.00           3        6.93   
3      Sandnessjøen     66.02      12.63        39.25          40        5.82   
4  Stann Creek Town     16.97     -88.23        85.37          20        9.17   

   Humidity Country        Date  
0        94      BR  1604012576  
1        76      PF  1604012626  
2        66      ZA  1604012638  
3        80      NO  1604012737  
4        74      BZ  1604012725

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [112]:
#set weight and locations
humidity = vacationDF["Humidity"]
locations = vacationDF[["Latitude", "Longitude"]]

#put onto gmap
mapG = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=5,point_radius=2)
mapG.add_layer(heat_layer)
mapG

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [113]:
#Narrow down to wind speed less than 10 mph, no cloudiness and a temperature more than 70 F but less than 80F
newVacation = vacationDF.loc[(vacationDF["Wind Speed"]<10)&(vacationDF["Cloudiness"]==0)&(vacationDF["Temperature"]>70)&(vacationDF["Temperature"]<80)].dropna()
newVacation

City  Latitude  Longitude  Temperature  Cloudiness  \
132              Pitimbu     -7.47     -34.81        78.80           0   
185       Victoria Point    -27.58     153.30        78.22           0   
228              Emerald    -23.53     148.17        75.90           0   
251                 Doka     13.52      35.77        79.02           0   
270            Quelimane    -17.88      36.89        70.77           0   
357  Santiago del Estero    -27.80     -64.26        70.47           0   
374           Alexandria     31.22      29.96        75.20           0   
407               Ormara     25.21      64.64        75.33           0   
426          Mitsamiouli    -11.38      43.28        78.44           0   
439            Morondava    -20.28      44.28        77.97           0   
454              Angoche    -16.23      39.91        76.21           0   
455              Sānchor     24.76      71.77        72.19           0   
476            Badarganj     25.67      89.05        70.57           0   
525               Nsanje    -16.92      35.26        73.00           0   
548                Aswān     24.09      32.91        75.24           0   
577                Kafue    -15.77      28.18        72.54           0   

     Wind Speed  Humidity Country        Date  
132        5.82        78      BR  1604012750  
185        1.99        74      AU  1604012756  
228        4.70        40      AU  1604012646  
251        2.73        31      SD  1604012764  
270        1.90        91      MZ  1604012766  
357        7.56        23      AR  1604012775  
374        8.05        60      EG  1604012777  
407        8.01        34      PK  1604012780  
426        8.12        78      KM  1604012782  
439        3.15        77      MG  1604012783  
454        3.02        84      MZ  1604012785  
455        2.13        29      IN  1604012785  
476        5.53        71      BD  1604012788  
525        4.32        59      MW  1604012630  
548        5.08        25      EG  1604012641  
577        5.77        49      ZM  1604012800

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [114]:
#new DF
hotel_df = pd.DataFrame(newVacation, columns=["City", "Country", "Latitude", "Longitude"])


#add column
hotel_df["Hotel Name"] = ""

hotel_df

City Country  Latitude  Longitude Hotel Name
132              Pitimbu      BR     -7.47     -34.81           
185       Victoria Point      AU    -27.58     153.30           
228              Emerald      AU    -23.53     148.17           
251                 Doka      SD     13.52      35.77           
270            Quelimane      MZ    -17.88      36.89           
357  Santiago del Estero      AR    -27.80     -64.26           
374           Alexandria      EG     31.22      29.96           
407               Ormara      PK     25.21      64.64           
426          Mitsamiouli      KM    -11.38      43.28           
439            Morondava      MG    -20.28      44.28           
454              Angoche      MZ    -16.23      39.91           
455              Sānchor      IN     24.76      71.77           
476            Badarganj      BD     25.67      89.05           
525               Nsanje      MW    -16.92      35.26           
548                Aswān      EG     24.09      32.91           
577                Kafue      ZM    -15.77      28.18

In [115]:
#loop to search for hotels within 5000 meters
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

for index, row in hotel_df.iterrows():
    
    lat = row["Latitude"]
    long = row["Longitude"]
    name = row["City"]

    params["location"] = f"{lat},{long}"

    print(f"Retrieving Results for Index {index}: {name}.")
    response = requests.get(url, params=params).json()
    results = response['results']
    
    try:
        print(f"Closest hotel in {name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    except (KeyError, IndexError):
        hotel_df.dropna(subset=['Hotel Name'], inplace = True)
        print("Closest hotel unavailable...")
    print("------------")


Retrieving Results for Index 132: Pitimbu.
Closest hotel unavailable...
------------
Retrieving Results for Index 185: Victoria Point.
Closest hotel in Victoria Point is Bay Retreat Motel.
------------
Retrieving Results for Index 228: Emerald.
Closest hotel in Emerald is Emerald Star Hotel.
------------
Retrieving Results for Index 251: Doka.
Closest hotel unavailable...
------------
Retrieving Results for Index 270: Quelimane.
Closest hotel in Quelimane is Hotel Elite.
------------
Retrieving Results for Index 357: Santiago del Estero.
Closest hotel in Santiago del Estero is Hotel Libertador.
------------
Retrieving Results for Index 374: Alexandria.
Closest hotel in Alexandria is Four Seasons Hotel Alexandria at San Stefano, Egypt.
------------
Retrieving Results for Index 407: Ormara.
Closest hotel in Ormara is Alqadri hotel zero point ormara.
------------
Retrieving Results for Index 426: Mitsamiouli.
Closest hotel in Mitsamiouli is Al Camar Lodge.
------------
Retrieving Results 

In [116]:
#display the available hotels
#drop ones without hotel
hotel_df.dropna(subset=['Hotel Name'], how = "any", inplace=True)
hotel_df

City Country  Latitude  Longitude  \
132              Pitimbu      BR     -7.47     -34.81   
185       Victoria Point      AU    -27.58     153.30   
228              Emerald      AU    -23.53     148.17   
251                 Doka      SD     13.52      35.77   
270            Quelimane      MZ    -17.88      36.89   
357  Santiago del Estero      AR    -27.80     -64.26   
374           Alexandria      EG     31.22      29.96   
407               Ormara      PK     25.21      64.64   
426          Mitsamiouli      KM    -11.38      43.28   
439            Morondava      MG    -20.28      44.28   
454              Angoche      MZ    -16.23      39.91   
455              Sānchor      IN     24.76      71.77   
476            Badarganj      BD     25.67      89.05   
525               Nsanje      MW    -16.92      35.26   
548                Aswān      EG     24.09      32.91   
577                Kafue      ZM    -15.77      28.18   

                                            Hotel Name  
132                                                     
185                                  Bay Retreat Motel  
228                                 Emerald Star Hotel  
251                                                     
270                                        Hotel Elite  
357                                   Hotel Libertador  
374  Four Seasons Hotel Alexandria at San Stefano, ...  
407                    Alqadri hotel zero point ormara  
426                                     Al Camar Lodge  
439                Palissandre Cote Ouest resort & SPA  
454                                 Guest House HESADA  
455                        HOTEL KAUSHAL INTERNATIONAL  
476                                                     
525                                       Shumba Lodge  
548                  Sofitel Legend Old Cataract Aswan  
577                    House Of Excellence Guest House

In [117]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [118]:
# Add marker layer ontop of heat map
mark = gmaps.marker_layer(locations, info_box_content = hotel_info)
mapG.add_layer(mark)

# Display figure
mapG

Figure(layout=FigureLayout(height='420px'))